In [1]:
import pandas as pd
import numpy as np


In [29]:
df = pd.read_csv('Sales_20250201_20250331.csv')
df.columns = df.columns.str.strip()
df.drop(columns=['Subtotal', 'Location', 'Staff Member', 'Income Category', 'Payer', 'Details'], inplace=True)
date_cols = ['Purchase Date', 'Invoice Date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
print(df.shape)
df.head()

(112, 9)


,Purchase Date,Invoice Date,Patient Guid,Item,Invoice #,Status,Total,Collected,Balance
0,2025-02-03,2025-02-03,29276-67,Individual Counselling Session (Video),1457-P01,paid,157.5,157.5,0.0
1,2025-02-03,2025-02-03,29276-79,Individual Counselling Session (Video),1458-P01,paid,157.5,157.5,0.0
2,2025-02-03,2025-02-03,29276-90,Individual Counselling Session (Video),1459-P01,paid,157.5,157.5,0.0
3,2025-02-03,2025-02-03,29276-40,Individual Counselling Session (Video),1460-P01,paid,157.5,157.5,0.0
4,2025-02-04,2025-02-04,29276-64,Individual Counselling Session (Video),1461-P01,paid,168.0,168.0,0.0


In [38]:
monthly_details = df.groupby(df['Purchase Date'].dt.to_period('M')).agg({'Invoice #': 'count', 'Total': 'sum', 'Collected':'sum', 'Balance': 'sum'}).reset_index()
monthly_details['Total_avg'] = round(monthly_details['Total'] / monthly_details['Invoice #'], 2)
monthly_details['Collected_avg'] = round(monthly_details['Collected'] / monthly_details['Invoice #'], 2)
monthly_details['Balance_avg'] = round(monthly_details['Balance'] / monthly_details['Invoice #'], 2)
monthly_details.columns = ['Month', '# of Sessions', 'Total Charged', 'Total Collected', 'Toal Outstanding', 'Average Charged', 'Average Collected', 'Average Outstanding']
monthly_details

,Month,# of Sessions,Total Charged,Total Collected,Toal Outstanding,Average Charged,Average Collected,Average Outstanding
0,2025-02,67,9978.75,9752.75,226.0,148.94,145.56,3.37
1,2025-03,45,6885.00,6465.00,420.0,153.00,143.67,9.33
